# Database examples for Scott

### Connect notebook to database

We'll use the ipython-sql modules for convenient access from the notebook, and an in-memory sqlite database named "scottdb".

In [ ]:
%load_ext sql
%sql sqlite:///scottdb:memory:

### Create database tables

In [ ]:
%%sql

DROP TABLE IF EXISTS GamePlays;
DROP TABLE IF EXISTS Games;

CREATE TABLE Games (
   ID INTEGER NOT NULL PRIMARY KEY,
   Game TEXT,
   Version TEXT,
   Barcode TEXT,
   MAC TEXT,
   IP TEXT,
   Location TEXT
);

CREATE TABLE GamePlays (
   GameID INTEGER REFERENCES Games,
   players INTEGER,
   p1_fire INTEGER,
   p2_fire INTEGER,
   playtime INTEGER,
   pwr_plays INTEGER,
   p1_pwr_plays INTEGER,
   p2_pwr_plays INTEGER,
   Timestamp DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP      
);

### Insert some data

This uses the "sqlalchemy" Python module which (more or less) automatically creates a bridge between Python objects and the database.

We insert a few example Games, and a bunch of random GamePlays.

In [ ]:
from random import *
from datetime import *

import sqlalchemy as sa

eng = sa.create_engine('sqlite:///scottdb:memory:')
meta = sa.MetaData()
meta.reflect(bind=eng)

Games = meta.tables['Games']
GamePlays = meta.tables['GamePlays']

exampleGames = [
    { 'ID':1, 'Game':'Some Game',    'Version':'1.0', 'Barcode':'1234567', 'MAC':'aabbccaabbcc', 'IP':'192.168.0.10', 'Location':'Garage'  },
    { 'ID':2, 'Game':'Another Game', 'Version':'1.1', 'Barcode':'2345678', 'MAC':'bbccaabbccaa', 'IP':'192.168.0.11', 'Location':'Garage'  },
    { 'ID':3, 'Game':'Cool Game',    'Version':'1.0', 'Barcode':'3456789', 'MAC':'ccaabbccaabb', 'IP':'192.168.0.12', 'Location':'Garage'  },
    { 'ID':4, 'Game':'Lame Game',    'Version':'2.0', 'Barcode':'4567890', 'MAC':'aaccbbaaccbb', 'IP':'192.168.1.10', 'Location':'Barcade' },
    { 'ID':5, 'Game':'Fave Game',    'Version':'3.0', 'Barcode':'5678901', 'MAC':'ccbbaaccbbaa', 'IP':'192.168.1.11', 'Location':'Barcade' }
]

with eng.connect() as con:
    con.execute(Games.insert().values(exampleGames))    
    for i in range(10000):
        con.execute(GamePlays.insert().values(
            GameID=randint(1,5),
            players=randint(1,2),
            playtime=randint(10,300),
            Timestamp=datetime(2021,1,1)+timedelta(days=randrange(365), minutes=randrange(24*60))))

### Inspect the generated data

In [ ]:
%sql SELECT * FROM Games

In [ ]:
%sql SELECT * FROM GamePlays LIMIT 10

## Query: plays per game on a given day

In [ ]:
%%sql
SELECT Game, COUNT(*) AS Plays
FROM Gameplays JOIN Games ON GamePlays.GameID = Games.ID
WHERE DATE(Timestamp) = '2021-05-01'
GROUP BY Game
ORDER BY Plays DESC

In [ ]:
_.DataFrame().plot.bar(x='Game', y='Plays', title="Plays on 2021-05-01")

## Query: Plays per game on a given day, playtime >= 20 seconds

In [ ]:
%%sql
SELECT Game, COUNT(*) AS Plays
FROM Gameplays JOIN Games ON GamePlays.GameID = Games.ID
WHERE DATE(Timestamp) = '2021-05-01' AND playtime >= 20
GROUP BY Game
ORDER BY Plays DESC

In [ ]:
_.DataFrame().plot.bar(x='Game', y='Plays', title="Plays (at least 20s) on 2021-05-01")

## Query: plays per game on a given week

In [ ]:
%%sql
SELECT Game, COUNT(*) AS Plays
FROM Gameplays JOIN Games ON GamePlays.GameID = Games.ID
WHERE strftime('%W', Timestamp) = '17'
GROUP BY Game
ORDER BY Plays DESC

In [ ]:
_.DataFrame().plot.bar(x='Game', y='Plays', title="Plays during 2021 week 17")

## Query: plays for particular game over a given month

In [ ]:
%%sql
SELECT CAST(strftime('%d', Timestamp) AS INT) as Day, COUNT(*) AS Plays
FROM Gameplays
WHERE GameID=5 AND strftime('%m', Timestamp) = '04'
GROUP BY day
ORDER BY day

In [ ]:
_.DataFrame().plot(x='Day', y='Plays', title='Plays of Fave Game during April 2021')